# Tree-structured Parzen estimators

In the lectures and the lab so far, we have used grid search to tune the hyperparameters of models. However, it is not a scalable strategy - the number of hyperparameter combinations increases exponentially with the number of hyperparameters.

In the last decade or so, there has been a lot of research in developing better search/optimization algorithms for hyperparameter optimization that require much less exploration of the hyperparameter space than grid search. We will look at one such method: the [Tree-structured Parzen Estimator (TPE)](https://papers.nips.cc/paper/2011/hash/86e8f7ab32cfd12577bc2619bc635690-Abstract.html) algorithm. While its often not the best performing hyperparameter optimization algorithm among recent advances across different settings, it is much more widely applicable, is much more easier to implement than the other algorithms, and can be easily scaled. The `hyperopt` library in Python contains an implementation of the TPE algorithm 

In this notebook, we will consider a simple example of tuning a single layer `MLPRegression` model on the concrete dataset. In the accompanying R script in the lab, we will use it to tune the hyperparameters of the `nnet` function. In the next lab, we use it for tuning gradient boosted trees.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

%matplotlib inline

plt.rcParams['figure.dpi'] = 150

## Read, load and preprocess the data

In [2]:
# load the concrete dataset
crt = pd.read_csv("../data/concrete.csv")

# standardize predictors
X = crt.drop('Strength',axis=1).values # extract as numpy ndarray
X_mean,X_std = X.mean(axis=0),X.std(axis=0)
X = (X-X_mean)/X_std

# standardize response
y = crt['Strength'].values
y_mean,y_std = y.mean(),y.std()
y = (y-y_mean)/y_std

## The TPE algorithm

In [3]:
# hyperopt imports
from hyperopt import hp, STATUS_OK,fmin,tpe, Trials, space_eval

# take care of convergence warnings
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

The first step is to define the search space. In the case of neural network regression model (with deterministic optimization), there are 3 hyperparameters.


1. `hidden_layer_sizes`: the number of units in the single hidden layer. This is an integer hyperparameter.
2. `alpha`: the L2 regularization hyperparameter. This is a continuous hyperarameter which we will optimize in the log-scale. 
3. `activation`: the choice of the activation function. One of `"relu"`, `"tanh"`, `"logistic"`. We chose a logistic activation function in Weeks 3 and 4. 

In [4]:
search_space = {
     # integer-valued hyperparameter that can take on values 2...40.
    'hidden_layer_sizes':hp.quniform('hidden_layer_sizes',2,40,q=1),
    # hyperparameter optimized in log-scale
    'alpha':hp.loguniform('alpha',np.log(1e-4),np.log(1)),
    # categorical hyperparameter,
    'activation':hp.choice('activation',['relu','tanh','logistic'])
}

The second step is to define a **loss** function. Here, we will use the 5-fold cross-validation root mean squared error (RMSE) as the loss function.

In [5]:
cv = KFold(5,random_state=1,shuffle=True)
train_test_splits = list(cv.split(X))


@ignore_warnings(category=ConvergenceWarning)
def cv_mse(config):
    K = cv.get_n_splits()
    rmse_folds = [None]*K
    
    for k,(train_index,test_index) in enumerate(train_test_splits):
        
        # fit a neural network model with the given configuration on the training data
        nn_fold = MLPRegressor(
            hidden_layer_sizes=int(config['hidden_layer_sizes']),
            activation = config['activation'],
            alpha = config['alpha'],
            # optimization settings
            solver='lbfgs',max_iter=1000
        ).fit(X[train_index,:],y[train_index])
        
        # obtain predictions on the holdout fold
        y_pred = nn_fold.predict(X[test_index,:])
        
        # compute RMSE
        rmse_folds[k] = np.sqrt(mean_squared_error(y[test_index],y_pred))
        
    
    return {'loss':np.mean(rmse_folds),'status':STATUS_OK} 

The third step is to use the `fmin` function to run the hyperopt algorithm. 

**Note**: You can get somewhat different results because `MLPRegressor` fits are noisy. 

In [6]:
trials = Trials() # maintains an optimization history
best = fmin(
    cv_mse, # loss function
    search_space, # search space
    algo=tpe.suggest,
    max_evals = 30, # number of hyperparameter configurations to explore
    trials=trials,
    return_argmin=False,
    rstate=np.random.default_rng(1)
)
best

100%|██████████| 30/30 [01:17<00:00,  2.58s/trial, best loss: 0.287045197187734]


{'activation': 'relu',
 'alpha': 0.016044828504314324,
 'hidden_layer_sizes': 26.0}

In [7]:
# gather all results
results = pd.DataFrame([
    space_eval(search_space,row.to_dict()) for _,row in pd.DataFrame(trials.vals).iterrows()
]) 

results['RMSE_CV'] = [tmp['loss'] for tmp in trials.results]
results['R2_cv'] = 1-results['RMSE_CV']**2/y.var()

results.sort_values('R2_cv',ascending=False).head(10)

,activation,alpha,hidden_layer_sizes,RMSE_CV,R2_cv
13,relu,0.016045,26.0,0.287045,0.917605
10,relu,0.009092,29.0,0.287388,0.917408
9,relu,0.015591,37.0,0.290107,0.915838
27,logistic,0.104334,19.0,0.301130,0.909321
21,relu,0.001993,40.0,0.303154,0.908097
29,relu,0.020123,23.0,0.307135,0.905668
3,relu,0.160314,33.0,0.308621,0.904753
8,tanh,0.025826,25.0,0.308811,0.904636
6,logistic,0.005094,25.0,0.309557,0.904175
16,tanh,0.000292,24.0,0.310537,0.903567


In [8]:
# fit final NN model
nn_final = MLPRegressor(
    hidden_layer_sizes=int(best['hidden_layer_sizes']),
    activation = best['activation'],
    alpha = best['alpha'],
    # optimization settings
    solver='lbfgs',max_iter=1000
).fit(X,y)

/Users/surajys/opt/anaconda3/envs/msia420-2/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:536: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
